In [ ]:
import numpy as np
from tqdm import tqdm
import re

# Load and preprocess

In [ ]:
# Load Shakespeare.txt
# Punctuation, including \n, included as separate "words"
# TODO: add syllable info
# TODO: HMM currently sucks, probably because I haven't done ^^^, and maybe because I need to do additional preprocessing somehow
# TODO: also easy to add rhyme (as in 'additional goals' section)
downloaded = drive.CreateFile({'id':'1cwaY0yRvUNgggFytJpca_1Mao85bawQN'})
downloaded.GetContentFile('filename.txt')

data = []
accum = []

f = open('filename.txt')

for line in f:
  if len(line) == 1:
    continue
  words = line.split()
  if len(words) == 1: # Line is only a number--beginning of new sonnet
    if len(accum) != 0:
      data.append(accum)
      accum = []
  else:
    for word in words:
      word = word.lower()
      # Case: no punctuation at the end
      if word[-1].isalpha(): 
        accum.append(word)
      # Case: last character is punctuation mark
      else:
        accum.append(word[:-1])
        accum.append(word[-1])
    accum.append('\n')

data = np.array([np.array(seq) for seq in data])
print(f'data.shape: {data.shape}')
print('')
print(f'data[0]: {data[0]}')

In [ ]:
# HMM expects sequences of integers, not strings

# mapping: character --> boolean array
mapping = {}
def get_mapping(k):
  v = mapping.get(k, len(mapping))
  if v == len(mapping):
    mapping[k] = v
  return v

X1 = np.array([np.array([get_mapping(k) for k in seq]) for seq in data])

mapping_rev = {v:k for k,v in mapping.items()}

print(f'X1.shape: {X1.shape}')
print('')
print(f'X1[0]: {X1[0]}')

In [ ]:
# hmmlearn in particular expects a single list of all observations, and a separate list of numbers indicating the length of each sequence
# Also, each observation should be a list
lengths = np.array([len(seq) for seq in X1])
X = np.concatenate(X1).reshape((-1, 1))
X,lengths

# HMM

In [ ]:
!pip install hmmlearn
from hmmlearn import hmm

In [ ]:
clfs = {i : hmm.MultinomialHMM(n_components=i, n_iter=1000) for i in range(5, 20)}
for i, clf in clfs.items():
  clf.fit(X, lengths)
  print(f'{i}: {clf.monitor_}')

In [ ]:
# Generate sonnet
for i, clf in clfs.items():
  sonnet, _ = clf.sample(len(data[1])) # Returns ([observations], [states])
  sonnet = sonnet.reshape(len(sonnet)) # Each observation is a list; extract it
  sonnet = ' '.join(mapping_rev[i] for i in sonnet)
  print(f'{i}: {sonnet}\n\n')